340 ms ± 8.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:

arg = []
sick = infect(s0,*arg)
dead = infect(s1,*arg)

In [3]:
#consistent colours
#notebook version top notch version
from bokeh.io import show, output_notebook, curdoc
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

import pandas as pd
import numpy as np
import datetime as dt
import requests
import json
#import psycopg2 
import os
import warnings
warnings.filterwarnings('ignore')

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, WidgetBox, column
from bokeh.models import HoverTool, Panel, NumeralTickFormatter, ColumnDataSource
from bokeh.models.widgets import CheckboxGroup, Tabs, Select, RadioButtonGroup


def modify(doc):
    def make_data(selection_sum, colour, norm = 0):

        '''
        Creates a column dataset based on interactive filter selection
        Calculates rolling average trenline for daily graphs.
        Returns ColumnDataSource.data.keys(): index, Date, Country, value series
        '''
        

        # create list of column names form selection
        ##remove 'No selection' from the list if there are others selected
        if len(selection_sum) >1 :
            selection_sum = [i for i in selection_sum if i != 'No selection']
        else : selection_sum = selection_sum

        sel_daily = [i+'_daily' for i in list(set(selection_sum))]
        sel_dead = [i+'_dead' for i in selection_sum]
        sel_daily_dead = [i+'_dead' for i in sel_daily]
        sel_daily_roll = [i+'_roll' for i in sel_daily]
        sel_daily_dead_roll = [i+'_roll' for i in sel_daily_dead]

        df = sick[selection_sum + sel_daily + sel_dead+ sel_daily_dead]
        
        # trendline
        days = 7
        for i in sel_daily:
            df[i+'_roll'] = df[i].rolling(window = days).mean()
            df[i+'_dead_roll'] = df[i+'_dead'].rolling(window = days).mean()
        
        #normalise with 100.000 population
        if norm != 0:
            try:
                selection_sum.remove('No selection')
            except: None

            temp = pd.DataFrame()
            for i in selection_sum:

                n = (pop['data'][i])
                temp1 = df.loc[:,[ii for ii in df.columns if i in ii]] / (n)
                temp = pd.concat((temp, temp1), axis = 1) 
                s_cds = temp
        else: s_cds = df.copy()

        s_cds.reset_index(inplace= True)

        s_cds['Date'] = pd.to_datetime(s_cds['Date'])

        #melts and combine sum values and diff values to a single df
        a1 = pd.melt(s_cds, id_vars = 'Date', value_vars = selection_sum, var_name = 'Country' )
        a2 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily, var_name = 'Country',value_name = 'value_day' )
        a3 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_dead, var_name = 'Country',value_name = 'value_dead' )
        a4 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_dead, var_name = 'Country',value_name = 'value_day_dead' )
        a5 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_roll, var_name = 'Country', value_name = 'value_day_roll')
        a6 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_dead_roll, var_name = 'Country', value_name = 'value_day_dead_roll')

        
        s_cds = pd.concat([a1,a2['value_day'],
                           a3['value_dead'],
                           a4['value_day_dead'],
                           a5['value_day_roll'],
                           a6['value_day_dead_roll']], axis = 1, ignore_index= True)
        s_cds.columns = ['Date', 'Country', 'value','value_day','value_dead','value_day_dead','value_day_roll','value_day_dead_roll']

        #get rid of empty dates except if 'No selection' is there
        s_cds = s_cds.applymap(lambda x: np.NaN if x == 0 else x)
        if 'No selection' not in s_cds['Country'].unique():
            s_cds.dropna(axis = 1, how = 'all', inplace = True)

        # colourise
        s_cds['colour'] = s_cds['Country'].map(colour)

        return ColumnDataSource(s_cds)

    def make_plot(src):
        '''
        Create plot according to fed in cds
        '''               
        hover_fig = {'Date': '@Date{%Y-%m-%d}'
                    ,'Country': '@Country'
                    ,'Infected': '@value{,000,000}'}
        hover_fig1 = {'Date': '@Date{%Y-%m-%d}'
                    ,'Country': '@Country'
                    ,'Infected': '@value_day{0}'}
        hover_fig2 = {'Date': '@Date{%Y-%m-%d}'
                    ,'Country': '@Country'
                    ,'Dead': '@value_dead{0}'}
        hover_fig3 = {'Date': '@Date{%Y-%m-%d}'
                    ,'Country': '@Country'
                    ,'Dead': '@value_day_dead{0}'}

        fig_kwargs = {
                      'plot_width': 700
                      ,'plot_height': 200
                      ,'x_axis_type': 'datetime'
                      ,'background_fill_color': '#f1f5fc'
                      ,'border_fill_color' : '#f1f5fc'

                      }
        plot_kwargs = {'source': src
                       ,'hover_fill_color':'red'
                       ,'hover_color':'red'
                       ,'hover_alpha': 1
                       ,'color' : 'colour'
                       ,'size': 6
                       ,'selection_color':'red'
                       ,'nonselection_color':'colour'
                      }
        line_kwargs = {'color' : 'black'
                       ,'source':src
                       ,'line_width': 0.5   
                        }

        fig = figure(**fig_kwargs 
                    ,title = 'Registered Infected'
                    ,toolbar_location = 'above'
                    ,tools = ['pan','wheel_zoom','box_zoom','box_select', 'reset'])
        fig.diamond(x='Date',y='value', **plot_kwargs, legend='Country')

        fig.add_tools(HoverTool(tooltips=hover_fig,
                                formatters ={'@Date': 'datetime', '@value': 'numeral'},
                                mode = 'vline'))
        fig.legend.location = 'top_left'
        fig.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig.ygrid.band_fill_color="olive"  #creates band colouring
        fig.ygrid.band_fill_alpha = 0.1
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig1 = figure(**fig_kwargs
                     ,title = 'Daily increase')
        fig1.diamond(x='Date',y='value_day', **plot_kwargs)
        fig1.vline_stack(x='Date', stackers='value_day_roll', **line_kwargs)
        fig1.x_range = fig.x_range
        fig1.add_tools(HoverTool(tooltips=hover_fig1,
                                formatters ={'@Date': 'datetime', '@value_day': 'numeral'},
                                mode = 'mouse'))
        fig1.legend.location = 'top_left'
        fig1.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig1.ygrid.band_fill_color="olive"  #creates band colouring
        fig1.ygrid.band_fill_alpha = 0.1
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig2 = figure(**fig_kwargs
                     ,title = 'Total dead')
        fig2.diamond(x='Date',y='value_dead', **plot_kwargs)
        fig2.x_range = fig.x_range
        fig2.add_tools(HoverTool(tooltips=hover_fig2,
                                formatters ={'@Date': 'datetime', '@value_dead': 'numeral'},
                                mode = 'vline'))
        fig2.legend.location = 'top_left'
        fig2.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig2.ygrid.band_fill_color="olive"  #creates band colouring
        fig2.ygrid.band_fill_alpha = 0.1            
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig3 = figure(**fig_kwargs
                     ,title = 'Daily increase dead')
        fig3.diamond(x='Date',y='value_day_dead', **plot_kwargs)
        fig3.line(x='Date', y = 'value_day_dead_roll', **line_kwargs)
        fig3.x_range = fig.x_range
        fig3.add_tools(HoverTool(tooltips=hover_fig3,
                                formatters ={'@Date': 'datetime', '@value_day_dead': 'numeral'},
                                mode = 'mouse'))
        fig3.legend.location = 'top_left'
        fig3.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig3.ygrid.band_fill_color="olive"  #creates band colouring
        fig3.ygrid.band_fill_alpha = 0.1                 



        layout1 =column(fig,fig1,fig2,fig3)
        layout2 =row(column(fig,fig1),column(fig2,fig3))
        return layout1,layout2

    def update(attr, old, new):

        global us, us2
        
        chkbox_act1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
        chkbox_act2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
        drpbox_act3 = [drop_box_group3.value]
        drpbox_act4 = [drop_box_group4.value]
        drpbox_act5 = [drop_box_group5.value]
        radiob_act6 = radiobuttgroup6.active
        selection_sum = chkbox_act1+chkbox_act2+drpbox_act3+drpbox_act4+drpbox_act5
        us = [chkbox_act1,chkbox_act2,drpbox_act3,drpbox_act4,drpbox_act5]
        selection_sum = list(set(selection_sum))
        
        #keeping existing colours
        colour_list = ['#1f77b4','#ff7f0e', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22','#17becf']
        old_colour = {i:c for i,c in zip(src.data['Country'],src.data['colour'])}
        colour = {}
        for i in selection_sum:
            if i in old_colour.keys():
                colour[i] = old_colour[i]
                colour_list.remove(old_colour[i])
        for i, n in enumerate(selection_sum):
            if n not in colour.keys():
                colour[n] = colour_list[i]
        
        new_src = make_data(selection_sum, colour, radiob_act6)            
        src.data.update(new_src.data)
        
        return src
     
    #data acquisition
    #sick = connect()
    sick = pd.read_csv('/home/trismegistos/Documents/DA/projectek/CoronaV_Countries/published/final_data.csv', index_col = 'Date')
    pop = json.load(open('population_covid.json'))
    col_list = [ i for i in sick.columns if 'dead' not in i and 'daily' not in i]
    colour_list = ['#1f77b4','#ff7f0e', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22','#17becf']

    #make selection list excuding checkbox values
    dropdown_list = ['No selection'] + [i for i in col_list \
                    if i not in ['No selection', 'World', 'China', 'US', 'United Kingdom','Korea, South','Italy']]

    checkbox_group1 = CheckboxGroup(labels=['World','China','US'], active=[0])
    checkbox_group2 = CheckboxGroup(labels=['United Kingdom','Italy','Korea, South'], active= [0])
    drop_box_group3 = Select(title="Select country",value='No selection',options=dropdown_list)
    drop_box_group4 = Select(title="Select country",value='No selection',options=dropdown_list)
    drop_box_group5 = Select(title="Select country",value='No selection',options=dropdown_list)
    radiobuttgroup6 = RadioButtonGroup(labels=['Absolute', 'Normalized'], active=1)

    checkbox_group1.on_change('active', update)
    checkbox_group2.on_change('active', update)
    drop_box_group3.on_change('value', update)
    drop_box_group4.on_change('value', update)
    drop_box_group5.on_change('value', update)
    radiobuttgroup6.on_change('active', update)


    ctrl1 = WidgetBox(checkbox_group1, sizing_mode = 'fixed', height= 80, width = 120) 
    ctrl2 = WidgetBox(checkbox_group2, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl3 = WidgetBox(drop_box_group3, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl4 = WidgetBox(drop_box_group4, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl5 = WidgetBox(drop_box_group5, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl6 = WidgetBox(radiobuttgroup6, sizing_mode = 'fixed', height= 80, width = 120)


    start_sel1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
    start_sel2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
    start_sel3 = [drop_box_group3.value]
    start_sel4 = [drop_box_group4.value]
    start_sel5 = [drop_box_group5.value]
    start_sel6 = radiobuttgroup6.active
    start_sum  = start_sel1 + start_sel2 +start_sel3 + start_sel4 + start_sel5
    colour = {'World':'#1f77b4', 'United Kingdom':'#ff7f0e'}
    
    src = make_data(start_sum, colour, start_sel6)
    fig = make_plot(src)

    coLayout = column(row(ctrl1,ctrl2, ctrl3, ctrl4,ctrl5, ctrl6),fig[0])
    roLayout = column(row(ctrl1,ctrl2, ctrl3, ctrl4,ctrl5, ctrl6),fig[1])
    colPanel = Panel(child = coLayout, title = '1x4 Layout')
    rowPanel = Panel(child = roLayout, title = '2x2 Layout')
    tabs = Tabs(tabs=[colPanel,rowPanel])
    
    doc.add_root(tabs)
    return doc


# Set up an application
output_notebook()
handler = FunctionHandler(modify)
app = Application(handler)

show(app)

Loading BokehJS ...

In [2]:
#rolling average
#notebook version top notch version
from bokeh.io import show, output_notebook, curdoc
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

import pandas as pd
import numpy as np
import datetime as dt
import requests
import json
#import psycopg2 
import os
import warnings
warnings.filterwarnings('ignore')

from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.layouts import row, WidgetBox, column
from bokeh.models import HoverTool, Panel, NumeralTickFormatter, ColumnDataSource
from bokeh.models.widgets import CheckboxGroup, Tabs, Select, RadioButtonGroup


def modify(doc):
    def make_data(selection_sum, norm = 0):
        '''
        Creates a column dataset based on interactive filter selection
        Calculates rolling average trenline for daily graphs.
        Returns ColumnDataSource.data.keys(): index, Date, Country, value series
        '''

        # create list of column names form selection
        ##remove 'No selection' from the list if there are others selected
        if len(selection_sum) >1 :
            selection_sum = [i for i in selection_sum if i != 'No selection']
        else : selection_sum = selection_sum

        sel_daily = [i+'_daily' for i in list(set(selection_sum))]
        sel_dead = [i+'_dead' for i in selection_sum]
        sel_daily_dead = [i+'_dead' for i in sel_daily]
        sel_daily_roll = [i+'_roll' for i in sel_daily]
        sel_daily_dead_roll = [i+'_roll' for i in sel_daily_dead]

        df = sick[selection_sum + sel_daily + sel_dead+ sel_daily_dead]
        
        # trendline
        days = 7
        for i in sel_daily:
            df[i+'_roll'] = df[i].rolling(window = days).mean()
            df[i+'_dead_roll'] = df[i+'_dead'].rolling(window = days).mean()

        #normalise with 100.000 population
        if norm != 0:
            try:
                selection_sum.remove('No selection')
            except: None

            temp = pd.DataFrame()
            for i in selection_sum:

                n = (pop['data'][i])
                temp1 = df.loc[:,[ii for ii in df.columns if i in ii]] / (n)
                temp = pd.concat((temp, temp1), axis = 1) 
                #print(temp1)
                s_cds = temp
        else: s_cds = df.copy()

        s_cds.reset_index(inplace= True)

        s_cds['Date'] = pd.to_datetime(s_cds['Date'])

        #melts and combine sum values and diff values to a single df
        a1 = pd.melt(s_cds, id_vars = 'Date', value_vars = selection_sum, var_name = 'Country' )
        a2 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily, var_name = 'Country',value_name = 'value_day' )
        a3 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_dead, var_name = 'Country',value_name = 'value_dead' )
        a4 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_dead, var_name = 'Country',value_name = 'value_day_dead' )
        a5 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_roll, var_name = 'Country', value_name = 'value_day_roll')
        a6 = pd.melt(s_cds, id_vars = 'Date', value_vars = sel_daily_dead_roll, var_name = 'Country', value_name = 'value_day_dead_roll')

        
        s_cds = pd.concat([a1,a2['value_day'],
                           a3['value_dead'],
                           a4['value_day_dead'],
                           a5['value_day_roll'],
                           a6['value_day_dead_roll']], axis = 1, ignore_index= True)
        s_cds.columns = ['Date', 'Country', 'value','value_day','value_dead','value_day_dead','value_day_roll','value_day_dead_roll']

        #get rid of empty dates exept if 'No selection' is there
        s_cds = s_cds.applymap(lambda x: np.NaN if x == 0 else x)
        if 'No selection' not in s_cds['Country'].unique():
            s_cds.dropna(axis = 1, how = 'all', inplace = True)

        # colourise
        colour = {i : c for i,c in zip(s_cds['Country'].unique(),colour_list)}
        s_cds['colour'] = s_cds['Country'].map(colour)

        return ColumnDataSource(s_cds)

    def make_plot(src):
        '''
        Create plot according to fed in cds
        '''               
        hover_fig = {'Date': '@Date{%Y-%m-%d}'
                         ,'Country': '@Country'
                        ,'Infected': '@value{,000,000}'}
        hover_fig1 = {'Date': '@Date{%Y-%m-%d}'
                         ,'Country': '@Country'
                        ,'Infected': '@value_day{0}'}
        hover_fig2 = {'Date': '@Date{%Y-%m-%d}'
                         ,'Country': '@Country'
                        ,'Dead': '@value_dead{0}'}
        hover_fig3 = {'Date': '@Date{%Y-%m-%d}'
                         ,'Country': '@Country'
                        ,'Dead': '@value_day_dead{0}'}

        fig_kwargs = {
                      'plot_width': 700
                      ,'plot_height': 200
                      ,'x_axis_type': 'datetime'
                      ,'background_fill_color': '#f1f5fc'
                      ,'border_fill_color' : '#f1f5fc'

                      }
        plot_kwargs = {'source': src
                       ,'hover_fill_color':'red'
                       ,'hover_color':'red'
                       ,'hover_alpha': 1
                       ,'color' : 'colour'
                       ,'size': 6
                       ,'selection_color':'red'
                       ,'nonselection_color':'colour'
                      }
        line_kwargs = {'color' : 'black'
                       ,'source':src
                       ,'line_width': 0.5   
                        }

        fig = figure(**fig_kwargs 
                    ,title = 'Registered Infected'
                    ,toolbar_location = 'above'
                    ,tools = ['pan','wheel_zoom','box_zoom','box_select', 'reset'])
        fig.diamond(x='Date',y='value', **plot_kwargs, legend='Country')

        fig.add_tools(HoverTool(tooltips=hover_fig,
                                formatters ={'@Date': 'datetime', '@value': 'numeral'},
                                mode = 'vline'))
        fig.legend.location = 'top_left'
        fig.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig.ygrid.band_fill_color="olive"  #creates band colouring
        fig.ygrid.band_fill_alpha = 0.1
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig1 = figure(**fig_kwargs
                     ,title = 'Daily increase')
        fig1.diamond(x='Date',y='value_day', **plot_kwargs)
        fig1.vline_stack(x='Date', stackers='value_day_roll', **line_kwargs)
        fig1.x_range = fig.x_range
        fig1.add_tools(HoverTool(tooltips=hover_fig1,
                                formatters ={'@Date': 'datetime', '@value_day': 'numeral'},
                                mode = 'vline'))
        fig1.legend.location = 'top_left'
        fig1.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig1.ygrid.band_fill_color="olive"  #creates band colouring
        fig1.ygrid.band_fill_alpha = 0.1
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig2 = figure(**fig_kwargs
                     ,title = 'Total dead')
        fig2.diamond(x='Date',y='value_dead', **plot_kwargs)
        fig2.x_range = fig.x_range
        fig2.add_tools(HoverTool(tooltips=hover_fig2,
                                formatters ={'@Date': 'datetime', '@value_dead': 'numeral'},
                                mode = 'vline'))
        fig2.legend.location = 'top_left'
        fig2.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig2.ygrid.band_fill_color="olive"  #creates band colouring
        fig2.ygrid.band_fill_alpha = 0.1            
      ##################<<<<<<<<<<<<>>>>>>>>>>#################  
        fig3 = figure(**fig_kwargs
                     ,title = 'Daily increase dead')
        fig3.diamond(x='Date',y='value_day_dead', **plot_kwargs)
        fig3.line(x='Date', y = 'value_day_dead_roll', **line_kwargs)
        fig3.x_range = fig.x_range
        fig3.add_tools(HoverTool(tooltips=hover_fig3,
                                formatters ={'@Date': 'datetime', '@value_day_dead': 'numeral'},
                                mode = 'vline'))
        fig3.legend.location = 'top_left'
        fig3.yaxis.formatter=NumeralTickFormatter(format=",000,000") #format number display
        fig3.ygrid.band_fill_color="olive"  #creates band colouring
        fig3.ygrid.band_fill_alpha = 0.1                 



        layout1 =column(fig,fig1,fig2,fig3)
        layout2 =row(column(fig,fig1),column(fig2,fig3))
        return layout1,layout2

    def update(attr, old, new):


        chkbox_act1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
        chkbox_act2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
        drpbox_act3 = [drop_box_group3.value]
        drpbox_act4 = [drop_box_group4.value]
        drpbox_act5 = [drop_box_group5.value]
        radiob_act6 = radiobuttgroup6.active
        print('Data type: {}'.format(radiob_act6))
        selection_sum = chkbox_act1+chkbox_act2+drpbox_act3+drpbox_act4+drpbox_act5

        selection_sum = list(set(selection_sum))
        print('Countries: {}'.format(selection_sum))
        new_src = make_data(selection_sum,radiob_act6)            
        src.data.update(new_src.data)

        return src
     
    #data acquisition
    #sick = connect()
    sick = pd.read_csv('/home/trismegistos/Documents/DA/projectek/CoronaV_Countries/published/final_data.csv', index_col = 'Date')
    pop = json.load(open('population_covid.json'))
    col_list = [ i for i in sick.columns if 'dead' not in i and 'daily' not in i]
    colour_list = ['#1f77b4','#ff7f0e', '#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22','#17becf']

    #make selection list excuding checkbox values
    dropdown_list = ['No selection'] + [i for i in col_list \
                    if i not in ['No selection', 'World', 'China', 'US', 'United Kingdom','Korea, South','Italy']]

    checkbox_group1 = CheckboxGroup(labels=['World','China','US'], active=[0])
    checkbox_group2 = CheckboxGroup(labels=['United Kingdom','Italy','Korea, South'], active= [0])
    drop_box_group3 = Select(title="Select country",value='No selection',options=dropdown_list)
    drop_box_group4 = Select(title="Select country",value='No selection',options=dropdown_list)
    drop_box_group5 = Select(title="Select country",value='No selection',options=dropdown_list)
    radiobuttgroup6 = RadioButtonGroup(labels=['Absolute', 'Normalized'], active=1)

    start_sel1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
    start_sel2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
    start_sel3 = [drop_box_group3.value]
    start_sel4 = [drop_box_group4.value]
    start_sel5 = [drop_box_group5.value]
    start_sel6 = radiobuttgroup6.active

    checkbox_group1.on_change('active', update)
    checkbox_group2.on_change('active', update)
    drop_box_group3.on_change('value', update)
    drop_box_group4.on_change('value', update)
    drop_box_group5.on_change('value', update)
    radiobuttgroup6.on_change('active', update)


    ctrl1 = WidgetBox(checkbox_group1, sizing_mode = 'fixed', height= 80, width = 120) 
    ctrl2 = WidgetBox(checkbox_group2, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl3 = WidgetBox(drop_box_group3, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl4 = WidgetBox(drop_box_group4, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl5 = WidgetBox(drop_box_group5, sizing_mode = 'fixed', height= 80, width = 120)
    ctrl6 = WidgetBox(radiobuttgroup6, sizing_mode = 'fixed', height= 80, width = 120)

    start_sum  = start_sel1 + start_sel2 +start_sel3 + start_sel4 + start_sel5 
    src = make_data(start_sum, start_sel6)

    fig = make_plot(src)

    coLayout = column(row(ctrl1,ctrl2, ctrl3, ctrl4,ctrl5, ctrl6),fig[0])
    roLayout = column(row(ctrl1,ctrl2, ctrl3, ctrl4,ctrl5, ctrl6),fig[1])
    colPanel = Panel(child = coLayout, title = '1x4 Layout')
    rowPanel = Panel(child = roLayout, title = '2x2 Layout')
    tabs = Tabs(tabs=[colPanel,rowPanel])
    
    doc.add_root(tabs)
    return doc


# Set up an application
output_notebook()
handler = FunctionHandler(modify)
app = Application(handler)

show(app)

Loading BokehJS ...

Data type: 1
Countries: ['United Kingdom', 'No selection']


ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'CheckboxGroup', 'id': '1574'}, 'attr': 'active', 'new': []}], 'references': []}: UnboundLocalError("local variable 's_cds' referenced before assignment")


Data type: 1
Countries: ['No selection']
Data type: 1
Countries: ['United Kingdom', 'No selection']
Data type: 1
Countries: ['No selection', 'United Kingdom', 'World']
Data type: 0
Countries: ['No selection', 'United Kingdom', 'World']
Data type: 1
Countries: ['No selection', 'United Kingdom', 'World']
Data type: 0
Countries: ['No selection', 'United Kingdom', 'World']
Data type: 1
Countries: ['No selection', 'United Kingdom', 'World']
